*Training Poker Using RLLib*

In [1]:
import sys
from poker_env import PokerEnv
from agents.random_policy import RandomActions
from agents.heuristic_policy import HeuristicPolicy
from agents.folder_policy import FolderPolicy
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.sac import SAC
from ray.rllib.algorithms.dqn import DQN
from ray.rllib.algorithms.appo import APPO
from ray.rllib.algorithms.r2d2 import R2D2
from gym import spaces
import mpu
import numpy as np
import ray
from ray import air, tune
from ray.rllib.models import MODEL_DEFAULTS
from ray.rllib.policy.policy import PolicySpec
from ray.tune.registry import register_env
from ray.tune import CLIReporter, register_env

/Users/aaronedwards/Documents/Projects/RL_Poker_Env/poker_env.py:10: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable


In Rllib, a policy function needs to be passed to map agent IDs to the policy to use. We also create and register the learning environment.

In [2]:
def select_policy(agent_id, episode, **kwargs):
    if agent_id == 0:
        return "learned"
    elif agent_id == 1:
        return "dqn"
    elif agent_id == 2:
        return "folder"
    elif agent_id == 3:
        return "random"
    return "Heuristic_100"

def env_creator(config):
    env = PokerEnv(select_policy, config)
    return env

register_env("poker", lambda config: env_creator(config))

The config describes all aspects of the training. A full list of the parameters is found here: https://github.com/ray-project/ray/blob/master/rllib/algorithms/algorithm_config.py 

In this example, we have used a default config that runs the PPO algorithm. Other heuristic agents have been defined that will play the game with the single learning agent. 

In [3]:
heuristic_observation_space = spaces.Dict({
            "hand": spaces.Box(0, 1, shape=(24, )),
            "community": spaces.Box(0, 1, shape=(24, ))
        })
action_space = spaces.Discrete(3)

#Defines the learning models architecture. 
model = MODEL_DEFAULTS.update({'fcnet_hiddens': [512, 512], 'fcnet_activation': 'relu', 'use_lstm': True})

config = (
    PPOConfig()
    #Each rollout worker uses a single cpu
    .rollouts(num_rollout_workers=8, num_envs_per_worker=1)\
    .training(train_batch_size=4000, gamma=0.99, model=model, lr=0.0004)\
    .environment(disable_env_checking=True)\
    .multi_agent(
        policies={
            #These policies thave pre-definded polices that dont learn.
            "random": PolicySpec(policy_class=RandomActions),
            "dqn": PolicySpec(config=DQN.get_default_config()),
            "folder": PolicySpec(policy_class=FolderPolicy),
            "Heuristic_100": (HeuristicPolicy, heuristic_observation_space, action_space, {'difficulty': 1}),
            #Passing nothing causes this agent to deafult to using a PPO policy
            "learned": PolicySpec(
                config=R2D2.get_default_config()
            ),
        },
        policy_mapping_fn=select_policy,
        policies_to_train=['learned', 'dqn'],
    )\
    .resources(num_gpus=0)\
    .framework('torch')
)
trainer = config.build(env="poker")

2022-11-10 16:31:59,345	WARNING ppo.py:351 -- `train_batch_size` (4000) cannot be achieved with your other settings (num_workers=8 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 500.
2022-11-10 16:32:01,836	INFO worker.py:1528 -- Started a local Ray instance.
(RolloutWorker pid=85625) /Users/aaronedwards/Documents/Projects/RL_Poker_Env/poker_env.py:10: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
(RolloutWorker pid=85625)   from collections import Iterable
(RolloutWorker pid=85622) /Users/aaronedwards/Documents/Projects/RL_Poker_Env/poker_env.py:10: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
(RolloutWorker pid=85622)   from collections import Iterable
(RolloutWorker pid=85620) /Users/aaronedwards/D

Training loop, each run will rollout x timesteps (where x is train_batch_size). An weight update is then applied using the rollout data. 

In [4]:
for i in range(1000):
    trainer.train()

2022-11-10 16:32:23,933	WARNING deprecation.py:47 -- DeprecationWarning: `_get_slice_indices` has been deprecated. This will raise an error in the future!
/Users/aaronedwards/opt/miniconda3/envs/poker2/lib/python3.9/site-packages/ray/rllib/algorithms/algorithm.py:1688: DeprecationWarning: invalid escape sequence \|
  """Adds a new policy to this Algorithm.


KeyboardInterrupt: 

In [5]:
#Saves a checkpoint of the trainer.
trainer.save("checkpoint/ppo_poker")

2022-11-11 09:18:31,059	WARNING policy.py:121 -- Can not figure out a durable policy name for <class 'agents.heuristic_policy.HeuristicPolicy'>. You are probably trying to checkpoint a custom policy. Raw policy class may cause problems when the checkpoint needs to be loaded in the future. To fix this, make sure you add your custom policy in rllib.algorithms.registry.POLICIES.
2022-11-11 09:18:31,063	WARNING policy.py:121 -- Can not figure out a durable policy name for <class 'agents.random_policy.RandomActions'>. You are probably trying to checkpoint a custom policy. Raw policy class may cause problems when the checkpoint needs to be loaded in the future. To fix this, make sure you add your custom policy in rllib.algorithms.registry.POLICIES.
2022-11-11 09:18:31,079	WARNING policy.py:121 -- Can not figure out a durable policy name for <class 'agents.folder_policy.FolderPolicy'>. You are probably trying to checkpoint a custom policy. Raw policy class may cause problems when the checkpoi

'checkpoint/ppo_poker/checkpoint_000719'